In [1]:
library(repr)
library(tidyverse)
library(tidymodels)
library(readxl)
options(repr.matrix.max.rows = 10)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [7]:
library(httr)
packageVersion("readxl")
# [1] ‘0.1.1’
url = "https://github.com/danizenarosa/dsci-100-grp43/raw/main/data/birds.xlsx"
GET(url, write_disk(tf <- tempfile(fileext = ".xlsx")))
df <- read_excel(tf, 2)
df

[1] ‘1.4.3’

Response [https://raw.githubusercontent.com/danizenarosa/dsci-100-grp43/main/data/birds.xlsx]
  Date: 2024-03-08 02:03
  Status: 200
  Content-Type: application/octet-stream
  Size: 21.5 MB
<ON DISK>  /tmp/Rtmp1kiqCT/file1c796f3c1083.xlsxNULL

Warning message:
“Expecting numeric in A4995 / R4995C1: got 'NA'”
Warning message:
“Expecting numeric in A5255 / R5255C1: got 'NA'”
Warning message:
“Expecting numeric in A5265 / R5265C1: got 'NA'”
Warning message:
“Expecting numeric in A5276 / R5276C1: got 'NA'”
Warning message:
“Expecting numeric in A6700 / R6700C1: got 'NA'”
Warning message:
“Expecting numeric in A6746 / R6746C1: got 'NA'”
Warning message:
“Expecting numeric in A7296 / R7296C1: got 'NA'”
Warning message:
“Expecting numeric in A7302 / R7302C1: got 'NA'”
Warning message:
“Expecting numeric in A7329 / R7329C1: got 'NA'”
Warning message:
“Expecting numeric in A7354 / R7354C1: got 'NA'”


Sequence,Species1,Family1,Order1,Avibase.ID1,Total.individuals,Female,Male,Unknown,Complete.measures,⋯,Habitat.Density,Migration,Trophic.Level,Trophic.Niche,Primary.Lifestyle,Min.Latitude,Max.Latitude,Centroid.Latitude,Centroid.Longitude,Range.Size
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
3103,Accipiter albogularis,Accipitridae,Accipitriformes,AVIBASE-BBB59880,5,2,0,3,4,⋯,1,2,Carnivore,Vertivore,Insessorial,-11.73,-4.0199999999999996,-8.15,158.49,37461.21
3090,Accipiter badius,Accipitridae,Accipitriformes,AVIBASE-1A0ECB6E,10,4,6,0,8,⋯,2,3,Carnivore,Vertivore,Insessorial,-29.47,46.39,8.23,44.98,22374973
3125,Accipiter bicolor,Accipitridae,Accipitriformes,AVIBASE-ADBE44E1,11,4,5,2,8,⋯,2,2,Carnivore,Vertivore,Generalist,-55.72,23.73,-10.1,-59.96,14309701.27
3116,Accipiter brachyurus,Accipitridae,Accipitriformes,AVIBASE-68BF920B,4,4,0,0,3,⋯,1,2,Carnivore,Vertivore,Insessorial,-6.31,-4.08,-5.45,150.68,35580.71
3092,Accipiter brevipes,Accipitridae,Accipitriformes,AVIBASE-8492E4B7,8,4,4,0,4,⋯,1,3,Carnivore,Vertivore,Generalist,31.19,55.86,45.24,45.33,2936751.8
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
3261,Trogon personatus,Trogonidae,Trogoniformes,AVIBASE-DEF06688,23,12,10,1,5,⋯,1,1,Omnivore,Omnivore,Insessorial,-20.399999999999999,11.16,-2.92,-71.92,745904.62
3256,Trogon rufus,Trogonidae,Trogoniformes,AVIBASE-EB24DFEE,22,10,12,0,8,⋯,1,1,Omnivore,Omnivore,Insessorial,-29.64,16.03,-5.73,-61.12,6666233.0199999996
3255,Trogon surrucura,Trogonidae,Trogoniformes,AVIBASE-8F2D3109,4,2,2,0,4,⋯,1,1,Carnivore,Invertivore,Insessorial,-31.18,-10.87,-23.08,-51.01,1366931


In [8]:
names(df)

[1] "Sequence"           "Species1"           "Family1"           
 [4] "Order1"             "Avibase.ID1"        "Total.individuals" 
 [7] "Female"             "Male"               "Unknown"           
[10] "Complete.measures"  "Beak.Length_Culmen" "Beak.Length_Nares" 
[13] "Beak.Width"         "Beak.Depth"         "Tarsus.Length"     
[16] "Wing.Length"        "Kipps.Distance"     "Secondary1"        
[19] "Hand-Wing.Index"    "Tail.Length"        "Mass"              
[22] "Mass.Source"        "Mass.Refs.Other"    "Inference"         
[25] "Traits.inferred"    "Reference.species"  "Habitat"           
[28] "Habitat.Density"    "Migration"          "Trophic.Level"     
[31] "Trophic.Niche"      "Primary.Lifestyle"  "Min.Latitude"      
[34] "Max.Latitude"       "Centroid.Latitude"  "Centroid.Longitude"
[37] "Range.Size"

In [ ]:
#bird_count <- group_by(bird_data, Family1) |>
#     summarize(count = n()) |>
#     arrange(count = desc(count)) |>
#     slice(1:10)
# bird_count